In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import numpy as np
from scipy.signal import welch

In [2]:
# !pip install imbalanced-learn

In [2]:
df = pd.read_csv('filtered_babydf8sec.csv')

In [3]:
# Analyzing class distribution
zero_count = (df.iloc[:, -1] == 0).sum()
one_count = (df.iloc[:, -1] == 1).sum()
print("Number of zeros:", zero_count)
print("Number of ones:", one_count)

Number of zeros: 42209
Number of ones: 5667


In [4]:
# Balance the classes by undersampling the majority class
df_class_0 = df[df.iloc[:, -1] == 0]
df_class_1 = df[df.iloc[:, -1] == 1]
df_class_0_balanced = df_class_0.sample(n=one_count, random_state=42) 
df_balanced = pd.concat([df_class_0_balanced, df_class_1])

In [ ]:
# Shuffle the balanced DataFrame
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [5]:
# Separate features and labels
X = df_balanced.iloc[:, :-1]
y = df_balanced.iloc[:, -1]
# Check new class distribution
print(df_balanced.iloc[:, -1].value_counts())

5376
0.0    5667
1.0    5667
Name: count, dtype: int64


In [6]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)  

In [7]:
# Encode the labels
y_encoded = to_categorical(y)

In [8]:
# Split the dataset 
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
# Define the model
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_scaled.shape[1], 1)),
    MaxPooling1D(2),
    Dropout(0.5),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Dropout(0.5),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(y_encoded.shape[1], activation='softmax')  
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val))




Epoch 1/30


284/284 [==============================] - 131s 450ms/step - loss: 0.8533 - accuracy: 0.5569 - val_loss: 0.6562 - val_accuracy: 0.6337
Epoch 2/30
284/284 [==============================] - 125s 440ms/step - loss: 0.6049 - accuracy: 0.6710 - val_loss: 0.5917 - val_accuracy: 0.7140
Epoch 3/30
284/284 [==============================] - 127s 448ms/step - loss: 0.5320 - accuracy: 0.7349 - val_loss: 0.5293 - val_accuracy: 0.7511
Epoch 4/30
284/284 [==============================] - 133s 468ms/step - loss: 0.4749 - accuracy: 0.7768 - val_loss: 0.5248 - val_accuracy: 0.7467
Epoch 5/30
284/284 [==============================] - 133s 468ms/step - loss: 0.4196 - accuracy: 0.8035 - val_loss: 0.5161 - val_accuracy: 0.7343
Epoch 6/30
284/284 [==============================] - 134s 473ms/step - loss: 0.3607 - accuracy: 0.8414 - val_loss: 0.4685 - val_accuracy: 0.7582
Epoch 7/30
284/284 [==============================] - 132s 465ms/step - loss: 0.2931 - accuracy: 0.8728 - val_loss: 0.4

In [10]:
train_acc = history.history['accuracy'][-1]  
val_acc = history.history['val_accuracy'][-1]  
print(f"Training Accuracy: {train_acc * 100:.2f}%")
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

Training Accuracy: 99.01%
Validation Accuracy: 81.55%


In [11]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

36/36 [==============================] - 2s 66ms/step - loss: 0.8218 - accuracy: 0.7901
Test Accuracy: 79.01%
